In [69]:
import pandas as pd

df = pd.read_csv(r"C:\Users\SRINATH\Desktop\data science\machine learing\ml project\data\raw\ultra_synthetic_200k.csv")
print("Rows:", len(df))
df.head()


Rows: 200000


,subject,body,answer,type,queue,priority,language,tag_1,action,ttr_hours,full_text,clean_text
0,System crash,"Hello team, I'm reporting a incident involving...",Our Technical Support team has reviewed your i...,Incident,Technical Support,high,en,urgent disruption,notify engineering team,1.98,"System crash Hello team, I'm reporting a incid...","system crash hello team, i'm reporting a incid..."
1,Authentication failure,"Hello team, I'm reporting a request involving ...",Our Technical Support team has reviewed your r...,Request,Technical Support,high,en,seeking support,verify user access,12.90,"Authentication failure Hello team, I'm reporti...","authentication failure hello team, i'm reporti..."
2,API timeout error,"Hello team, I'm reporting a change involving a...",Our Technical Support team has reviewed your c...,Change,Technical Support,medium,en,apply migration,validate change request,4.84,"API timeout error Hello team, I'm reporting a ...","api timeout error hello team, i'm reporting a ..."
3,System crash,"Hello team, I'm reporting a change involving a...",Our Technical Support team has reviewed your c...,Change,Technical Support,low,en,apply migration,validate change request,9.92,"System crash Hello team, I'm reporting a chang...","system crash hello team, i'm reporting a chang..."
4,System crash,"Hello team, I'm reporting a change involving a...",Our Technical Support team has reviewed your c...,Change,Technical Support,low,en,apply migration,review configuration update,26.07,"System crash Hello team, I'm reporting a chang...","system crash hello team, i'm reporting a chang..."


In [70]:
df.columns

Index(['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language',
       'tag_1', 'action', 'ttr_hours', 'full_text', 'clean_text'],
      dtype='object')

In [71]:
# common vectorizer


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,2))
X = vectorizer.fit_transform(df["clean_text"])


MODEL 1 — TEAM CLASSIFIER (queue)

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

y = df["queue"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model_team = LinearSVC()
model_team.fit(X_train, y_train)

pred = model_team.predict(X_test)

print("TEAM Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


TEAM Accuracy: 1.0
                                 precision    recall  f1-score   support

           Billing and Payments       1.00      1.00      1.00      4000
               Customer Service       1.00      1.00      1.00      4000
                General Inquiry       1.00      1.00      1.00      4000
                Human Resources       1.00      1.00      1.00      4000
                     IT Support       1.00      1.00      1.00      4000
                Product Support       1.00      1.00      1.00      4000
          Returns and Exchanges       1.00      1.00      1.00      4000
            Sales and Pre-Sales       1.00      1.00      1.00      4000
Service Outages and Maintenance       1.00      1.00      1.00      4000
              Technical Support       1.00      1.00      1.00      4000

                       accuracy                           1.00     40000
                      macro avg       1.00      1.00      1.00     40000
                   weighted av

In [74]:
# MODEL 2 — PRIORITY CLASSIFIER 

In [75]:
y = df["priority"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model_priority = LinearSVC()
model_priority.fit(X_train, y_train)

pred = model_priority.predict(X_test)

print("PRIORITY Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


PRIORITY Accuracy: 0.331475
              precision    recall  f1-score   support

        high       0.33      0.33      0.33     13364
         low       0.33      0.33      0.33     13273
      medium       0.33      0.33      0.33     13363

    accuracy                           0.33     40000
   macro avg       0.33      0.33      0.33     40000
weighted avg       0.33      0.33      0.33     40000



In [76]:
# MODEL 3 — TYPE CLASSIFIER

TYPE Accuracy: 1.0
              precision    recall  f1-score   support

      Change       1.00      1.00      1.00      9994
    Incident       1.00      1.00      1.00     10042
     Problem       1.00      1.00      1.00      9956
     Request       1.00      1.00      1.00     10008

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000



In [78]:
# MODEL 4 — ACTION CLASSIFIER

In [79]:
y = df["action"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model_action = LinearSVC()
model_action.fit(X_train, y_train)

pred = model_action.predict(X_test)

print("ACTION Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


ACTION Accuracy: 0.33395
                                   precision    recall  f1-score   support

           apply diagnostic steps       0.32      0.33      0.33      3311
           approve access request       0.33      0.34      0.34      3347
check logs for recurring patterns       0.34      0.33      0.34      3317
          notify engineering team       0.33      0.32      0.32      3329
      perform root-cause analysis       0.33      0.33      0.33      3329
        provide requested details       0.34      0.34      0.34      3323
         restart affected service       0.34      0.34      0.34      3364
      review configuration update       0.33      0.33      0.33      3332
       schedule deployment window       0.33      0.34      0.34      3362
        trigger incident protocol       0.33      0.34      0.34      3349
          validate change request       0.33      0.33      0.33      3300
               verify user access       0.34      0.33      0.34      3337

In [80]:
# MODEL 5 — TTR REGRESSION (hours)

In [82]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

y = df["ttr_hours"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

regressor = Ridge()
regressor.fit(X_train, y_train)

pred = regressor.predict(X_test)

print("MAE (hours):", mean_absolute_error(y_test, pred))


MAE (hours): 7.730376599928827


In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\Users\SRINATH\Desktop\data science\machine learing\ml project\data\raw\ultra_synthetic_200k_v2.csv")
print(df.shape)
print(df['type'].value_counts())
print(df['priority'].value_counts())
print(df['action'].value_counts().head(10))
print(df['ttr_hours'].describe())
# Inspect a few examples to verify action & priority phrases are inside clean_text
for i in range(5):
    print("----")
    print(df.loc[i, ['queue','type','priority','action']])
    print(df.loc[i,'clean_text'][:300])


(200000, 12)
type
Problem     50250
Incident    50247
Request     49957
Change      49546
Name: count, dtype: int64
priority
medium    66744
low       66667
high      66589
Name: count, dtype: int64
action
notify engineering team              16917
check logs for recurring patterns    16779
review configuration update          16768
perform root-cause analysis          16756
restart affected service             16717
apply diagnostic steps               16715
verify user access                   16707
provide requested details            16673
trigger incident protocol            16613
approve access request               16577
Name: count, dtype: int64
count    200000.000000
mean         17.954250
std          18.984559
min           1.000000
25%           2.980000
50%          10.060000
75%          23.980000
max          72.000000
Name: ttr_hours, dtype: float64
----
queue              Technical Support
type                        Incident
priority                         low
action

In [ ]:
df.columns

Index(['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language',
       'tag_1', 'action', 'ttr_hours', 'full_text', 'clean_text'],
      dtype='object')

In [ ]:
df.head(1)

,subject,body,answer,type,queue,priority,language,tag_1,action,ttr_hours,full_text,clean_text
0,System crash,"Hello team, I'm reporting a incident involving...",Thank you for contacting Technical Support. We...,Incident,Technical Support,low,en,complete outage,restart affected service,2.68,"System crash Hello team, I'm reporting a incid...","system crash hello team, i'm reporting a incid..."


In [ ]:
def predict_ticket(subject, body, vectorizer,
                   model_team, model_priority, model_type, model_action, ttr_model):
    """
    Run full ML pipeline prediction.
    TEAM → PRIORITY → TYPE → ACTION → TTR (hours)
    """

    # Build text (same cleaning you used in training)
    full_text = (subject + " " + body).lower()

    # Vectorize
    X_vec = vectorizer.transform([full_text])

    # ---- PREDICTIONS ----
    team_pred = model_team.predict(X_vec)[0]
    priority_pred = model_priority.predict(X_vec)[0]
    type_pred = model_type.predict(X_vec)[0]
    action_pred = model_action.predict(X_vec)[0]
    ttr_pred = float(regressor.predict(X_vec)[0])

    # Return structured output
    return {
        "team": team_pred,
        "priority": priority_pred,
        "type": type_pred,
        "action": action_pred,
        "ttr_hours": round(ttr_pred , 2)
    }


In [ ]:
result = predict_ticket(
    subject="Payment failure after scheduled upgrade",
    body="""Hello team, after last night’s upgrade, several customers 
            cannot complete payments. I already validated API keys and 
            retried payments but issue persists.""",

    vectorizer=vectorizer,
    model_team=model_team,
    model_priority=model_priority,
    model_type=model_type,
    model_action=model_action,
    ttr_model=regressor
)

print(result)


{'team': 'Technical Support', 'priority': 'medium', 'type': 'Incident', 'action': 'restart affected service', 'ttr_hours': 13.2}


In [ ]:
test_cases = [
    {
        "subject": "Customers unable to complete payments",
        "body": """Hello team, after yesterday’s deployment several users reported 
                    that payment attempts are failing with a “transaction not processed” error.  
                    I already validated API keys and retried via backup gateway, but failures continue.  
                    Please check urgently."""
    },
    {
        "subject": "Confusion about analytics dashboard",
        "body": """Hi support, I’m having trouble understanding the analytics dashboard. 
                    Several panels are blank and I’m unsure if it's a configuration issue 
                    or missing permissions. Please help clarify."""
    },
    {
        "subject": "VPN stopped working after patch",
        "body": """This is unacceptable. My VPN stopped working after your latest security patch.  
                    I can’t access any internal systems and this is blocking my work.  
                    Fix this immediately."""
    },
    {
        "subject": "Incorrect salary credited",
        "body": """Hello HR, I received a lower salary amount this month. 
                    This seems like a payroll calculation error.  
                    Please verify and correct the issue."""
    },
    {
        "subject": "Returning damaged item",
        "body": """Hello team, the product I received is damaged.  
                    I want to initiate a return and request replacement.  
                    Please guide me with the next steps."""
    },
    {
        "subject": "Need detailed pricing for enterprise plan",
        "body": """Hi, we are evaluating your enterprise solution and need 
                    a detailed pricing breakdown for 200 users.  
                    Please send the quotation."""
    },
    {
        "subject": "Database crash during peak load",
        "body": """Our production DB crashed twice today during high traffic.  
                    Logs show connection pool exhaustion.  
                    We urgently need support to stabilize the environment."""
    },
    {
        "subject": "Need info about your data retention policy",
        "body": """Hello team, could you provide details on your data retention 
                    and deletion policy for customer records?"""
    },
    {
        "subject": "Multiple users unable to login",
        "body": """We are seeing widespread login failures across our teams.  
                    Tried resetting passwords and clearing cache but issue continues."""
    },
    {
        "subject": "Need configuration update in production",
        "body": """Please schedule a configuration update for our production environment.  
                    We need new routing rules deployed before Friday."""
    }
]


In [ ]:
for i, case in enumerate(test_cases, start=1):
    result = predict_ticket(
        subject=case["subject"],
        body=case["body"],
        vectorizer=vectorizer,
        model_team=model_team,
        model_priority=model_priority,
        model_type=model_type,
        model_action=model_action,
        ttr_model=regressor
    )

    print(f"\n=== TEST {i} RESULT ===")
    print(result)



=== TEST 1 RESULT ===
{'team': 'Billing and Payments', 'priority': 'high', 'type': 'Problem', 'action': 'check logs for recurring patterns', 'ttr_hours': 13.25}

=== TEST 2 RESULT ===
{'team': 'Product Support', 'priority': 'medium', 'type': 'Request', 'action': 'verify user access', 'ttr_hours': 18.7}

=== TEST 3 RESULT ===
{'team': 'IT Support', 'priority': 'high', 'type': 'Request', 'action': 'verify user access', 'ttr_hours': 25.36}

=== TEST 4 RESULT ===
{'team': 'Human Resources', 'priority': 'medium', 'type': 'Problem', 'action': 'verify user access', 'ttr_hours': 20.45}

=== TEST 5 RESULT ===
{'team': 'Returns and Exchanges', 'priority': 'high', 'type': 'Problem', 'action': 'apply diagnostic steps', 'ttr_hours': 17.43}

=== TEST 6 RESULT ===
{'team': 'Sales and Pre-Sales', 'priority': 'high', 'type': 'Change', 'action': 'validate change request', 'ttr_hours': 13.96}

=== TEST 7 RESULT ===
{'team': 'Service Outages and Maintenance', 'priority': 'high', 'type': 'Change', 'action